In [1]:
from platform import python_version

print(python_version())

3.7.5


In [156]:
import os
import numpy as np
from os import listdir
from PIL import Image
import time
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from keras import backend as K
import matplotlib.pyplot as plt

In [157]:
path="datasets/ofg_family/"
randomiser = np.random.RandomState(123)

In [158]:
def generate_image(family_dir):
    dic={}
    sub=[a for a in listdir(path+"/"+family_dir)]
    
    for ele in sub:
        mypath = path+"/"+family_dir+"/"+ele+"/"
        onlyfiles = [mypath+f for f in listdir(mypath)]
        
        addr = randomiser.choice(onlyfiles)
        original_img = np.array(Image.open(addr).resize((32,32),Image.ANTIALIAS))
        if ele[0].lower()=='f':
            dic['father'] = original_img
        elif ele[0].lower()=='m':
            dic['mother'] = original_img
        elif ele.lower()=='child_male':
            dic['child'] = original_img    
            dic['gender']=np.zeros((original_img.shape))
        elif ele.lower()=='child_female':
            dic['child'] = original_img    
            dic['gender'] = np.ones((original_img.shape))
    return [dic['father'],dic['mother'],dic['gender'],dic['child']]

def generate_batch(families_batch):
    np_images=[]
    
    for family in families_batch:
        res = generate_image(family)
        if( res != None):
            np_images.append(res)
    
    return np_images

In [169]:
#full_data = load_npdata(path)
#print(len(full_data))

for r, d, f in os.walk(path):
        all_families=d
        break

# randomiser.shuffle(all_families)

train_families = all_families[:-500]
test_families = all_families[-500:]

create encoder decoder supervised pipeling operation
input: stacked images of father and mother (32,32,6)
encoder -> decoder -> image reconstruction
reconstruction loss w/ target image as child

In [170]:
concat = tf.keras.layers.Concatenate()

family_data = generate_image(all_families[0])
inp = concat([family_data[0],family_data[1]])
child = family_data[3]
child = tf.convert_to_tensor(child, dtype=tf.float32)


In [171]:
OUTPUT_CHANNELS = 3

In [172]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [173]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [174]:
def EncoderNN():
  down_stack = [
    downsample(32, 4, apply_batchnorm=True),
    downsample(64, 4, apply_batchnorm=False),
    downsample(128, 4, apply_batchnorm=False) 
  ]

  up_stack = [
    upsample(64, 4, apply_dropout=False),
    upsample(32, 4, apply_dropout=False),
    upsample(3, 4, apply_dropout=False)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)

  concat = tf.keras.layers.Concatenate()

  inputs = tf.keras.layers.Input(shape=[32,32,6])
  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = concat([x, skip])
    
  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [175]:
encoder = EncoderNN()

# gen_output = encoder(inp[tf.newaxis,...], training=False)
# loss= tf.reduce_mean(tf.image.total_variation(tf.concat([child,tf.squeeze(gen_output)],axis=0)))
# loss = tf.reduce_mean(tf.nn.l2_loss(child-gen_output))
# print(loss)

In [176]:
def custom_loss():
    def loss(y_true,y_pred):
        print(y_true.shape)
        print(y_pred.shape)
        return tf.reduce_mean(tf.abs(y_true - y_pred))
        #return tf.reduce_mean(tf.image.total_variation(tf.concat([y_true,y_pred],axis=0)))
    return loss

inp = concat([full_data[0][0],full_data[0][1]])
inp = tf.convert_to_tensor(inp, dtype=tf.float32)
X_train = tf.reshape(inp,([1,32,32,6]))
Y_train = tf.reshape(full_data[0][3],(1,32,32,3))

encoder.compile(loss=custom_loss(),
              optimizer=tf.keras.optimizers.RMSprop())
history = encoder.fit(X_train, Y_train,
                    batch_size=1,
                    epochs=1)

gen_output = encoder(inp[tf.newaxis,...], training=False)

In [177]:
concat = tf.keras.layers.Concatenate()

In [178]:
encoder.compile(loss=custom_loss(),optimizer=tf.keras.optimizers.RMSprop())

divide = 100
epochs = 5
batch = 20

for epoch in range(epochs):
    print("Epoch ", epoch , " .....")
    for i in range(len(train_families)//batch):
        batch_data = np.asarray(generate_batch(train_families[i*batch:(i+1)*batch]))
        
        print("Generated batch", batch_data.shape)
        
        X_train = tf.convert_to_tensor(concat([batch_data[:,0],batch_data[:,1]]),dtype =tf.float32)
        print("Batch converted to tensor")
        
        Y_train = batch_data[:,3]
        history = encoder.fit(X_train, Y_train, batch_size=batch)

(None, None, None, None)
(None, 32, 32, 3)
Epoch  0  .....
Generated batch (20, 4, 32, 32, 3)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Batch converted to tensor
Train on 20 samples
(20, 32, 32, 3)
(20, 32, 32, 3)
(20, 32, 32, 3)
(20, 32, 32, 3)
20/20 [==============================] - 1s 41ms/sample - loss: 116.4446
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 135.9211
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 sampl

20/20 [==============================] - 0s 3ms/sample - loss: 128.3428
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 125.6830
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 131.6009
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 129.8153
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 147.1551
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 133.7550
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 120.6073
Generated 

20/20 [==============================] - 0s 2ms/sample - loss: 127.0999
Epoch  1  .....
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 123.0626
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 130.6590
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 132.1517
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 123.3848
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 119.5200
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 126

Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 139.3452
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 130.5103
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 121.2901
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 119.8727
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 125.0928
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 131.8267
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
2

20/20 [==============================] - 0s 2ms/sample - loss: 111.5906
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 123.0674
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 127.7501
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 127.3259
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 123.7957
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 128.4425
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 127.0982
Generated 

Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 131.9574
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 127.5965
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 119.6753
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 123.8756
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 133.7922
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 128.7283
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
2

20/20 [==============================] - 0s 3ms/sample - loss: 125.7360
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 135.1809
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 127.5106
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 121.3074
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 125.9248
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 125.4619
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 139.3625
Generated 

Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 132.8757
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 119.9371
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 118.6049
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 126.0725
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 133.6463
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 139.8833
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
2

Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 134.3100
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 128.0557
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 120.7602
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 134.2583
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 3ms/sample - loss: 124.0210
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 116.1791
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
2

Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 132.1335
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 134.8318
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 131.4920
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 135.0706
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 119.4688
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
20/20 [==============================] - 0s 2ms/sample - loss: 127.8981
Generated batch (20, 4, 32, 32, 3)
Batch converted to tensor
Train on 20 samples
2